In [3]:
import pandas as pd
import os

In [ ]:
# Avergae GCP per district
# Number of at_risks students per district
# Engagement rate per district


# Extract District Information 
# 001 Washington District
# 002 New York District

# Create a table for districts
#distric_id
# district_name

In [4]:
# Define file paths
raw_data = '../data/raw/'
processed_data = '../data/processed/'



In [96]:
def create_district_mapping(filepath):
    filename = filepath.split('/')[-1].split(".")[0]

    district_mapping = {
        'school_alpha': 'Washington District',
        'school_beta': 'New York District',
        'school_gamma': 'New York District'
    }

     
    return district_mapping[filename]
    
    

In [6]:
def clean_data(file_path):

    df = pd.read_csv(file_path)
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    
    column_map = {
    'id': 'student_id',
    'full_name': 'full_name',
    'sex': 'gender',
    'maths': 'math_score',
    'english_language': 'english_score',
    'sci': 'science_score',
    'attendance_%': 'attendance_rate',
    'mathscore': 'math_score',
    'englishscore': 'english_score',
    'english_score': 'english_score',
    'science_score': 'science_score',
    'attendance': 'attendance_rate',
    'Sex': 'gender',
    'Maths': 'math_score'
}

    df.rename(columns=column_map, inplace=True)


    df['math_score'] = pd.to_numeric(df['math_score'], errors='coerce')
    df['english_score'] = pd.to_numeric(df['english_score'], errors='coerce')
    df['science_score'] = pd.to_numeric(df['science_score'], errors='coerce')
    df['attendance_rate'] = pd.to_numeric(df['attendance_rate'], errors='coerce')

    return df[['student_id', 'full_name', 'gender', 'math_score',
                'english_score', 'science_score', 'attendance_rate', 'term']]

In [7]:
file_names = ['school_alpha.csv', 'school_beta.csv', 'school_gamma.csv']
raw_files = [os.path.join(raw_data, f) for f in file_names]




In [47]:
more_data = []

for file in raw_files:
    clean_file = clean_data(file)
    clean_file['district'] = create_district_mapping(file)
    
    more_data.append(clean_file)
all_data = pd.concat(more_data, ignore_index=True)





In [ ]:
# exports new york and washington districts to their respective csv files 

nyc = all_data[all_data['district' ] == 'New York District']
dc = all_data[all_data['district' ] == 'Washington District']

nyc.to_csv(os.path.join(processed_data, '002_New_York_District.csv'), index=False)
dc.to_csv(os.path.join(processed_data, '001_Washington_District.csv'), index=False)

### exports all the data with districts to a single csv file 

In [ ]:

all_data.to_csv(os.path.join(processed_data, 'schools_with_district.csv'), index=False); 



### Reads at risk csv 

In [101]:
at_risk = pd.read_csv(os.path.join(processed_data, 'at_risk_students.csv'))

####  Merges the district data with at risk data

In [102]:



mgdata = pd.merge(at_risk, all_data[['student_id','math_score','english_score','science_score','district','attendance_rate']], on='student_id'); mgdata



,student_id,full_name,gpa,risk_score,engagement_level,math_score,english_score,science_score,district,attendance_rate
0,5506-350,Mary Smith,59.000000,50,Low,92.0,14.0,71.0,Washington District,74.98
1,7873-325,Patricia Moore,39.000000,51,Low,1.0,87.0,29.0,Washington District,60.82
2,3615-814,Robert Smith,47.333333,50,Low,63.0,59.0,20.0,Washington District,68.49
3,2584-467,Mary Miller,63.333333,50,Low,58.0,41.0,91.0,Washington District,71.65
4,2654-489,Mary Jones,19.333333,51,Low,38.0,17.0,3.0,Washington District,66.82
...,...,...,...,...,...,...,...,...,...,...
320,4522-955,Elizabeth Smith,53.666667,50,Low,29.0,46.0,86.0,New York District,60.08
321,6828-505,Mary Jones,60.333333,50,Low,83.0,81.0,17.0,New York District,61.62
322,3848-131,Robert Brown,53.333333,50,Low,93.0,32.0,35.0,New York District,64.58
323,5093-196,William Moore,28.000000,51,Low,18.0,27.0,39.0,New York District,74.76


# Number of at_risks students per district

In [ ]:
# number of at risk students per district 
def calculate_at_risk_students(data=mgdata):
    
    at_risk_counts = data.groupby('district')['risk_score'].value_counts().reset_index()

    at_risk_counts.rename(columns={'risk_score': 'at_risk_students'}, inplace=True)

    return at_risk_counts
    
    

calculate_at_risk_students()

,district,at_risk_students,count
0,New York District,50,98
1,New York District,1,74
2,New York District,51,43
3,Washington District,50,46
4,Washington District,1,42
5,Washington District,51,22


# Engagement rate per district

In [87]:
def calculate_engagement_rate():
    engagement_rate = mgdata.groupby('district')['engagement_level'].value_counts().reset_index()
    
    return engagement_rate



calculate_engagement_rate()

,district,engagement_level,count
0,New York District,Low,141
1,New York District,Medium,42
2,New York District,High,32
3,Washington District,Low,68
4,Washington District,High,23
5,Washington District,Medium,19


## AVERAGE GCP 

In [94]:
def calculate_average_gcp(data=all_data):
    data['GCP'] = data[['math_score', 'english_score', 'science_score']].mean(axis=1)

    # Group by district and calculate average GCP
    average_gcp_per_district = data.groupby('district')['GCP'].mean().reset_index()

    average_gcp_per_district = average_gcp_per_district.sort_values(by='GCP', ascending=False)
    

    
    return average_gcp_per_district

calculate_average_gcp(mgdata)

,district,GCP
0,New York District,41.906202
1,Washington District,41.903030


In [95]:
mgdata

,student_id,full_name,gpa,risk_score,engagement_level,math_score,english_score,science_score,district,attendance_rate,GCP
0,5506-350,Mary Smith,59.000000,50,Low,92.0,14.0,71.0,Washington District,74.98,59.000000
1,7873-325,Patricia Moore,39.000000,51,Low,1.0,87.0,29.0,Washington District,60.82,39.000000
2,3615-814,Robert Smith,47.333333,50,Low,63.0,59.0,20.0,Washington District,68.49,47.333333
3,2584-467,Mary Miller,63.333333,50,Low,58.0,41.0,91.0,Washington District,71.65,63.333333
4,2654-489,Mary Jones,19.333333,51,Low,38.0,17.0,3.0,Washington District,66.82,19.333333
...,...,...,...,...,...,...,...,...,...,...,...
320,4522-955,Elizabeth Smith,53.666667,50,Low,29.0,46.0,86.0,New York District,60.08,53.666667
321,6828-505,Mary Jones,60.333333,50,Low,83.0,81.0,17.0,New York District,61.62,60.333333
322,3848-131,Robert Brown,53.333333,50,Low,93.0,32.0,35.0,New York District,64.58,53.333333
323,5093-196,William Moore,28.000000,51,Low,18.0,27.0,39.0,New York District,74.76,28.000000


,student_id,full_name,gpa,risk_score,engagement_level,math_score,english_score,science_score,district,attendance_rate
0,5506-350,Mary Smith,59.000000,50,Low,92.0,14.0,71.0,Washington District,74.98
1,7873-325,Patricia Moore,39.000000,51,Low,1.0,87.0,29.0,Washington District,60.82
2,3615-814,Robert Smith,47.333333,50,Low,63.0,59.0,20.0,Washington District,68.49
3,2584-467,Mary Miller,63.333333,50,Low,58.0,41.0,91.0,Washington District,71.65
4,2654-489,Mary Jones,19.333333,51,Low,38.0,17.0,3.0,Washington District,66.82
...,...,...,...,...,...,...,...,...,...,...
320,4522-955,Elizabeth Smith,53.666667,50,Low,29.0,46.0,86.0,New York District,60.08
321,6828-505,Mary Jones,60.333333,50,Low,83.0,81.0,17.0,New York District,61.62
322,3848-131,Robert Brown,53.333333,50,Low,93.0,32.0,35.0,New York District,64.58
323,5093-196,William Moore,28.000000,51,Low,18.0,27.0,39.0,New York District,74.76
